### РОССИЙСКИЙ УНИВЕРСИТЕТ ДРУЖБЫ НАРОДОВ

#### Факультет физико-математических и естественных наук  

## ОТЧЕТ ПО ЛАБОРАТОРНОЙ РАБОТЕ № 2


### Дисциплина: Управление ИТ-сервисами и контентом

##### Студент:  Матюхин Григорий Васильевич
##### Группа:   НПИбд-01-21

## Москва 2024
***

# Введение

В данной лабораторной работе изучается web-сервис [random.org](https://random.org). Данный веб-сервис
предоставляет API для получения последовательностей истинно случайных чисел. Этот сервис работает по протоколу [JSON-RPC](https://www.jsonrpc.org/).

## JSON-RPC

Что-бы вызвать какую-то процедуру на сервере, работающем на JSON-RPC, клиенту необходимо отправить на правильный эндпоинт объект, содержащий следующие поля:
- jsonrpc -- версия протокола, всегда равно "2.0";
- method -- название метода, который следует вызвать;
- params -- значения аргументов к методу;
- id -- идентификатор запроса.

В ответ на это сервер отправит объект из:
- jsonrpc -- так же как в запросе;
- result -- если успешно выполнено, результат выполнения метода;
- error -- если была ошибка, описание ошибки;
- id -- то же самое значение, как и id в запросе. Клиент может использовать это для корреляции запросов и ответов.

# Выполнение

Для доступа к API необходим ключ. Его можно получить зарегестрировавшись на сайте.
Сохраним его в переменную.

In [12]:
RANDOMORG_KEY=XXX

## Генерация целых чисел

### Простая генерация пяти целых чисел

Сформируем запрос на генерацию пяти целых чисел в диапазоне $[-10, 50]$.
Для этого вызовем метод `generateIntegers` с необходимыми параметрами:
- n -- количество чисел;
- min -- минимум;
- max -- максимум;

Выведем тело запроса и ответа (помечены `>` и `<` соответственно),
а так-же и сам ответ в формате JSON.

In [2]:
BASE_URL=https://api.random.org/json-rpc/4/invoke

In [3]:
(curl -s -X POST $BASE_URL \
  -H "Content-Type: application/json" \
  -v \
  -d @- <<END
  {
    "jsonrpc": "2.0",
    "id": 69420,
    "method": "generateIntegers",
    "params": {
      "apiKey": "$RANDOMORG_KEY",
      "n": 5,
      "min": -10,
      "max": 50
    }
  }
END
) \
  2> >(rg "<|>") \
  > >(jq)

> POST /json-rpc/4/invoke HTTP/2
> Host: api.random.org
> User-Agent: curl/8.7.1
> Accept: */*
> Content-Type: application/json
> Content-Length: 196
> 
< HTTP/2 200 
< date: Tue, 30 Apr 2024 20:04:36 GMT
< content-type: application/json; charset=utf-8
< content-length: 185
< x-content-type-options: nosniff
< access-control-allow-origin: *
< access-control-allow-headers: Origin, X-Requested-With, Content-Type, Accept
< cf-cache-status: DYNAMIC
< set-cookie: __cflb=02DiuEJLVpaAQLxvFoC1yWv9tVL87GjjiASj16m9YzVg7; SameSite=Lax; path=/; expires=Tue, 07-May-24 20:04:36 GMT; HttpOnly
< strict-transport-security: max-age=31536000; includeSubDomains; preload
< server: cloudflare
< cf-ray: 87ca244beaa878ff-DME
< 
{
  "jsonrpc": "2.0",
  "result": {
    "random": {
      "data": [
        -5,
        45,
        -8,
        16,
        4
      ],
      "completionTime": "2024-04-30 20:04:36Z"
    },
    "bitsUsed": 30,
    "bitsLeft": 247876,
    "requestsLeft": 948,
    "advisoryDelay": 1440
  }

В теле ответа получаем следующую информацию:

- jsonrpc -- версия протокола
- result -- результат выполнения процедуры
  - random -- случайные числа
    - data -- данные
    - completionTime -- когда был осуществлен запрос
  - bitsUsed -- сколько битов случайности требовалось для выполнения
  - bitsLeft -- сколько битов осталось в квоте для данного ключа
  - requestsLeft -- сколько запросов осталось
  - advisoryDelay -- сколько времени рекомендуется ждать перед следующим запросом, чтобы не превысить rate limit
- id -- id запроса

Рассмотрим заголовки ответа:
- date -- дата
- content-type -- тип данных ответа
- content-length -- длинна данных ответа
- x-content-type-options -- браузер не должен догадываться, какой тип данных был отправлен, поле content-type верно
- access-control-allow-origin -- кому можно посылать запросы
- access-control-allow-headers -- разрешить запросы с этими заголовками
- cf-cache-status -- надо ли кэшировать эту страницу в Cloudflare CDN
- set-cookie -- установить куки с данным значением
- strict-transport-security -- гарантия, что сайт, влкючая поддоменны, будет доступен по HTTPS как минимум год
- server -- был использован прокси Cloudflare
- cf-ray -- идентификатор в логах Cloudflare

### Генерация типов в восьмеричной системе

Можно сгенерировать числа в другой системе исчисления.
Для этого необходимо задать опциональный параметр `base`.

In [4]:
(curl -s -X POST $BASE_URL \
  -H "Content-Type: application/json" \
  -d @- <<END
  {
    "jsonrpc": "2.0",
    "id": 69420,
    "method": "generateIntegers",
    "params": {
      "apiKey": "$RANDOMORG_KEY",
      "n": 5,
      "min": -10,
      "max": 50,
      "base": 8
    }
  }
END
) | jq

{
  "jsonrpc": "2.0",
  "result": {
    "random": {
      "data": [
        "25",
        "30",
        "41",
        "32",
        "07"
      ],
      "completionTime": "2024-04-30 20:04:37Z"
    },
    "bitsUsed": 30,
    "bitsLeft": 247846,
    "requestsLeft": 947,
    "advisoryDelay": 1440
  },
  "id": 69420
}


Теперь числа в ответе приходят как JSON-строки, а не как JSON-числа.

## Ошибки

Если метод в JSON-RPC не может вернуть результат, то он возвращает ошибку.
В теле есть код ошибки, строка с человеко-читаемым объяснением ошибки и дополнительные данные,
чтобы ее исправить.

### Неверный JSON

Поставим лишнюю запятую в запросе.

In [5]:
(curl -s -X POST $BASE_URL \
  -H "Content-Type: application/json" \
  -d @- <<END
  {
    "jsonrpc": "2.0",
    "id": 69420,
    "method": "generateIntegers",
    "params": {
      "apiKey": "$RANDOMORG_KEY",
      "n": 5,
      "min": -10,
      "max": 50,
    }
  }
END
) | jq

{
  "jsonrpc": "2.0",
  "error": {
    "code": -32700,
    "message": "Parse error",
    "data": null
  },
  "id": null
}


### Метод не найден

Попробуем вызвать несуществующий метод.

In [6]:
(curl -s -X POST $BASE_URL \
  -H "Content-Type: application/json" \
  -d @- <<END
  {
    "jsonrpc": "2.0",
    "id": 69420,
    "method": "generateInteger",
    "params": {
      "apiKey": "$RANDOMORG_KEY",
      "n": 5,
      "min": -10,
      "max": 50
    }
  }
END
) | jq

{
  "jsonrpc": "2.0",
  "error": {
    "code": -32601,
    "message": "Method not found",
    "data": null
  },
  "id": 69420
}


### Плохие параметры процедуры

Запросим больше чисел чем есть в диапазоне.

In [7]:
(curl -s -X POST $BASE_URL \
  -H "Content-Type: application/json" \
  -d @- <<END
  {
    "jsonrpc": "2.0",
    "id": 69420,
    "method": "generateIntegers",
    "params": {
      "apiKey": "$RANDOMORG_KEY",
      "n": 5,
      "min": 1,
      "max": 4,
      "replacement": false
    }
  }
END
) | jq

{
  "jsonrpc": "2.0",
  "error": {
    "code": 301,
    "message": "You requested 5 values without replacement but the domain you specified only contains 4",
    "data": [
      5,
      4
    ]
  },
  "id": 69420
}


## Генерация числовых последовательностей

Используем метод `generateIntegerSequences`. В параметрах:
- n -- количество последовательнойстей;
- min -- минимум;
- max -- максимум;
- length -- длинна последовательностей.

In [8]:
(curl -s -X POST $BASE_URL \
  -H "Content-Type: application/json" \
  -d @- <<END
  {
    "jsonrpc": "2.0",
    "id": 69420,
    "method": "generateIntegerSequences",
    "params": {
      "apiKey": "$RANDOMORG_KEY",
      "n": 5,
      "min": -10,
      "max": 50,
      "length": 3
    }
  }
END
) | jq

{
  "jsonrpc": "2.0",
  "result": {
    "random": {
      "data": [
        [
          5,
          36,
          25
        ],
        [
          30,
          8,
          33
        ],
        [
          -7,
          -8,
          -6
        ],
        [
          -2,
          5,
          21
        ],
        [
          -3,
          12,
          19
        ]
      ],
      "completionTime": "2024-04-30 20:04:41Z"
    },
    "bitsUsed": 90,
    "bitsLeft": 247756,
    "requestsLeft": 946,
    "advisoryDelay": 1410
  },
  "id": 69420
}


Теперь `result.random.data` содержит список списков случайных чисел.

## Использование

### 1d6

In [9]:
(curl -s -X POST $BASE_URL \
  -H "Content-Type: application/json" \
  -d @- <<END
  {
    "jsonrpc": "2.0",
    "id": 69420,
    "method": "generateIntegers",
    "params": {
      "apiKey": "$RANDOMORG_KEY",
      "n": 1,
      "min": 1,
      "max": 6
    }
  }
END
) | jq '.result.random.data[0]'

4


### Подброс монеты

- 1 -- орел
- 0 -- решка

In [10]:
(curl -s -X POST $BASE_URL \
  -H "Content-Type: application/json" \
  -d @- <<END
  {
    "jsonrpc": "2.0",
    "id": 69420,
    "method": "generateIntegers",
    "params": {
      "apiKey": "$RANDOMORG_KEY",
      "n": 1,
      "min": 0,
      "max": 1
    }
  }
END
) | jq '.result.random.data[0]'

1


### 5 IPv4 адресов

In [11]:
(curl -s -X POST $BASE_URL \
  -H "Content-Type: application/json" \
  -d @- <<END
  {
    "jsonrpc": "2.0",
    "id": 69420,
    "method": "generateIntegerSequences",
    "params": {
      "apiKey": "$RANDOMORG_KEY",
      "n": 5,
      "length": 4,
      "min": 0,
      "max": 255 
    }
  }
END
) | jq '.result.random.data.[] | join(".")'

"153.20.106.0"
"119.88.89.108"
"44.94.152.241"
"95.37.73.30"
"194.11.24.131"
